## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

В столбцах **days_employed** (общий трудовой стаж в днях) и **total_income** (ежемесячный доход) имеются пропуски.<br/>
Столбец **days_employed** (общий трудовой стаж в днях) содержит дробные и отрицательные значения.<br/>
Столбец **total_income** (ежемесячный доход) содержит дробные значения.<br/>
Столбец **education** (уровень образования клиента) содержит строчные значения в разных регистрах.

### Шаг 2. Предобработка данных

### Обработка пропусков

Посчитаем количество пропущенных значений в столбцах **days_employed** и **total_income** и взглянем на строки с пропущенными значениями

In [4]:
days_emp_na = data['days_employed'].isna().sum()
tot_inc_na = data['total_income'].isna().sum()
total_rows = data.shape[0] # всего строк
print('Количество строк с пропущнными значенями в столбце days_employed:', days_emp_na)
print('Количество строк с пропущнными значенями в столбце total_income:', tot_inc_na)
print('Доля строк с пропущенными значениями {:.2%}'.format(days_emp_na/total_rows))

Количество строк с пропущнными значенями в столбце days_employed: 2174
Количество строк с пропущнными значенями в столбце total_income: 2174
Доля строк с пропущенными значениями 10.10%


In [5]:
data_na = data[(data['days_employed'].isna()) & (data['total_income'].isna())]
print("Строк с пропущенными значениями по двум столбцам одновременно:", data_na.shape[0])
data_na.groupby('income_type').count()

Строк с пропущенными значениями по двум столбцам одновременно: 2174


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,debt,total_income,purpose
income_type,,,,,,,,,,,
госслужащий,147,0,147,147,147,147,147,147,147,0,147
компаньон,508,0,508,508,508,508,508,508,508,0,508
пенсионер,413,0,413,413,413,413,413,413,413,0,413
предприниматель,1,0,1,1,1,1,1,1,1,0,1
сотрудник,1105,0,1105,1105,1105,1105,1105,1105,1105,0,1105


**Посмотрим как распределены пропущенные значения по каждому столбцу отдельно**

In [6]:
data_na['children'].value_counts() / data['children'].value_counts() * 100

-1      6.382979
 0     10.170330
 1      9.858863
 2      9.927007
 3     10.909091
 4     17.073171
 5     11.111111
 20    11.842105
Name: children, dtype: float64

In [7]:
data_na['dob_years'].value_counts() / data['dob_years'].value_counts() * 100

0      9.900990
19     7.142857
20     9.803922
21    16.216216
22     9.289617
23    14.173228
24     7.954545
25     6.442577
26     8.578431
27     7.302231
28    11.332008
29     9.174312
30    10.740741
31    11.607143
32     7.254902
33     8.777969
34    11.442786
35    10.372771
36    11.351351
37     9.869646
38     9.030100
39     8.900524
40    10.837438
41     9.719934
42    10.887772
43     9.746589
44     8.043876
45    10.060362
46    10.105263
47    12.291667
48     8.550186
49     9.842520
50     9.922179
51    11.160714
52    10.950413
53     9.586057
54    11.482255
55    10.835214
56    11.088296
57    12.173913
58    12.147505
59     7.657658
60    10.344828
61    10.704225
62    10.795455
63    10.780669
64    13.962264
65    10.309278
66    10.928962
67     9.580838
68     9.090909
69     5.882353
70     4.615385
71     8.620690
72     6.060606
73    12.500000
74          NaN
75          NaN
Name: dob_years, dtype: float64

In [8]:
data_na['education_id'].value_counts() / data['education_id'].value_counts() * 100

0    10.342205
1    10.109630
2     9.274194
3     7.446809
4          NaN
Name: education_id, dtype: float64

In [9]:
data_na['family_status_id'].value_counts() / data['family_status_id'].value_counts() * 100

0     9.991922
1    10.581757
4    10.238180
3     9.372385
2     9.895833
Name: family_status_id, dtype: float64

In [10]:
data_na['gender'].value_counts() / data['gender'].value_counts() * 100

F      10.424276
M       9.467618
XNA          NaN
Name: gender, dtype: float64

In [11]:
data_na['income_type'].value_counts() / data['income_type'].value_counts() * 100

безработный              NaN
в декрете                NaN
госслужащий        10.075394
компаньон           9.990167
пенсионер          10.710581
предприниматель    50.000000
сотрудник           9.937944
студент                  NaN
Name: income_type, dtype: float64

In [12]:
data_na['debt'].value_counts() / data['debt'].value_counts() * 100

0    10.129397
1     9.764503
Name: debt, dtype: float64

In [13]:
data_na['purpose'].value_counts() / data['purpose'].value_counts() * 100

автомобили                                11.924686
автомобиль                                 8.282828
высшее образование                         8.830022
дополнительное образование                10.389610
жилье                                      9.273570
заняться высшим образованием              11.290323
заняться образованием                     13.349515
на покупку автомобиля                      6.355932
на покупку подержанного автомобиля         8.768267
на покупку своего автомобиля              10.495050
на проведение свадьбы                     11.840412
недвижимость                               9.779180
образование                                9.395973
операции с жильем                         11.332312
операции с коммерческой недвижимостью     10.752688
операции с недвижимостью                   9.023669
операции со своей недвижимостью           11.269841
покупка жилой недвижимости                10.049423
покупка жилья                              8.037094
покупка жиль

**Промежуточный вывод** <br>
Во всех строках всегда пропущены значения в обоих столбцах.<br>
Можно заметить что отношение по разным столбцам в среднем имеет значение 10%, можно сказать. Можно сказать что пропуски полностю случайные<br>
Доля пропусков по всей таблице также равна 10%. Это значение довольно большое, значит эти строки нельзя удалить.

In [14]:
print('Минимальное значение в столбце days_employed:', data['days_employed'].min())
print('Максимальное значение в столбце days_employed:', data['days_employed'].max())
print('Количество строк с отрицательными значениями в days_employed:', data[data['days_employed'] < 0]['days_employed'].count(), '\n')
print('Минимальное значение в столбце total_income:', data['total_income'].min())
print('Максимальное значение в столбце total_income:', data['total_income'].max())


Минимальное значение в столбце days_employed: -18388.949900568383
Максимальное значение в столбце days_employed: 401755.40047533
Количество строк с отрицательными значениями в days_employed: 15906 

Минимальное значение в столбце total_income: 20667.26379327158
Максимальное значение в столбце total_income: 2265604.028722744


Судя по значениями в столбце **days_employed**: его неободимо предварительно привести в порядок (взять модуль), вероятно 15900 отрицательных значений - неправильная работа программы, сформировавшая этот отчет. А также надо будет исключить записи, в которых указан нереальный трудовой стаж, отбирать будем по условию: <br><br>
*Трудовой стаж < (Возраст клиента - 16) * 365* <br><br>
16 - возраст с которого можно официально работать<br>
365 - умножаем на это число, чтобы привести к одной размерности с трудовым стажем 

In [15]:
data['days_employed'] = data['days_employed'].apply(abs) # Берем модуль
data_new = data[ (data['days_employed'] < (data['dob_years'] - 16 ) * 365) & (data['days_employed'] > 0) ] # Получаем таблицу с реальными значениями трудового стажа
days_employed_mean = data_new['days_employed'].mean()
total_income_median = data_new['total_income'].median()
data['days_employed'] = data['days_employed'].fillna(days_employed_mean)
data['total_income'] = data['total_income'].fillna(total_income_median)
data = data[ (data['days_employed'] < (data['dob_years'] - 16 ) * 365) & (data['days_employed'] > 0) ]

print('Минимальное значение в столбце days_employed:', data['days_employed'].min())
print('Среднее по days_employed:', days_employed_mean)
print('Медианное по total_income:', total_income_median)
print()
data.info()

Минимальное значение в столбце days_employed: 24.14163324048118
Среднее по days_employed: 2315.568776439035
Медианное по total_income: 151157.91831966495

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17866 entries, 0 to 21524
Data columns (total 12 columns):
children            17866 non-null int64
days_employed       17866 non-null float64
dob_years           17866 non-null int64
education           17866 non-null object
education_id        17866 non-null int64
family_status       17866 non-null object
family_status_id    17866 non-null int64
gender              17866 non-null object
income_type         17866 non-null object
debt                17866 non-null int64
total_income        17866 non-null float64
purpose             17866 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 1.8+ MB


Также есть одна строка с неизвестными данными в столбце **gender**, так как она одна, ее можно удалить 

In [16]:
data = data[data['gender'] != 'XNA']

In [17]:
print('Количество строк с отрицательным значением в столбце children {:.5%}'.format(len(data[data['children'] < 0]) / len(data)))
data = data[data['children'] >= 0]

Количество строк с отрицательным значением в столбце children 0.22390%


### Вывод

В столбцах **days_employed** и **total_income** содержались пропуски типа NaN.<br>
Пропуски в столбцах полностью случайные. Так есть закономерность: пропуск всегда в двух столбцах сразу и не зависит от типа занятости, вероятно ошибка в программе, формирующей файл. <br>
Пропуски заполнены средним для **days_employed** и медианным для **total_income**, спомощью метода **fillna()**. Для **total_income** было выбрано медианное значение, потому что максимальный значение дохода может являться большим выбросом и будет сильно смещать среднее значение. Предварительно столбец **days_employed** был преобразован: <br>
Т.к. было много отрицательных значений (вероятно ошибка программы, формирующей файл), надо было взять их модуль. Затем отбросить строки, в которых значение **days_employed** было нереальным, т.е. трудовой стаж превышал возраст клиента.<br>
Была удалена строка с пропущенным полем **gender**<br>
Были строки с отрицательным значением в столбце **children**, но не известно: означало отрицательное значение отсутствие детей или просто ошибка в знаке, т.е. не ясно: надо было присвоить им значение 0 или взять модуль. Так как доля таких строк составляла менее 1%, они были удалены

### Замена типа данных

In [18]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17825 entries, 0 to 21524
Data columns (total 12 columns):
children            17825 non-null int64
days_employed       17825 non-null int64
dob_years           17825 non-null int64
education           17825 non-null object
education_id        17825 non-null int64
family_status       17825 non-null object
family_status_id    17825 non-null int64
gender              17825 non-null object
income_type         17825 non-null object
debt                17825 non-null int64
total_income        17825 non-null int64
purpose             17825 non-null object
dtypes: int64(7), object(5)
memory usage: 1.8+ MB


### Вывод

Для того чтобы преобразовать данные в нужный тип использовали метод **astype()**

### Обработка дубликатов

In [19]:
print('Количество дубликатов:', data.duplicated().sum())
data[data.duplicated()]

Количество дубликатов: 54


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,2315,41,среднее,1,женат / замужем,0,F,сотрудник,0,151157,покупка жилья для семьи
4182,1,2315,34,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,151157,свадьба
4851,0,2315,60,среднее,1,гражданский брак,1,F,пенсионер,0,151157,свадьба
5557,0,2315,58,среднее,1,гражданский брак,1,F,пенсионер,0,151157,сыграть свадьбу
7808,0,2315,57,среднее,1,гражданский брак,1,F,пенсионер,0,151157,на проведение свадьбы
8583,0,2315,58,высшее,0,Не женат / не замужем,4,F,пенсионер,0,151157,дополнительное образование
9238,2,2315,34,среднее,1,женат / замужем,0,F,сотрудник,0,151157,покупка жилья для сдачи
9528,0,2315,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,151157,операции со своей недвижимостью
9627,0,2315,56,среднее,1,женат / замужем,0,F,пенсионер,0,151157,операции со своей недвижимостью
10462,0,2315,62,среднее,1,женат / замужем,0,F,пенсионер,0,151157,покупка коммерческой недвижимости


Помимо полностью одинаковых строк, необходимо привести столбцы строчного типа к одному регистру и проверить на дупликаты еще раз

In [20]:
data['education'].value_counts()

среднее                11074
высшее                  4186
неоконченное высшее      632
СРЕДНЕЕ                  631
Среднее                  554
ВЫСШЕЕ                   243
Высшее                   243
начальное                166
Неоконченное высшее       44
НЕОКОНЧЕННОЕ ВЫСШЕЕ       27
Начальное                 11
НАЧАЛЬНОЕ                 10
ученая степень             3
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [21]:
data['family_status'].value_counts()

женат / замужем          10374
гражданский брак          3544
Не женат / не замужем     2458
в разводе                  978
вдовец / вдова             471
Name: family_status, dtype: int64

In [22]:
data['gender'].value_counts()

F    11247
M     6578
Name: gender, dtype: int64

In [23]:
data['income_type'].value_counts()

сотрудник          10948
компаньон           5028
госслужащий         1436
пенсионер            409
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [24]:
data['purpose'].value_counts()

свадьба                                   668
на проведение свадьбы                     643
сыграть свадьбу                           634
операции с недвижимостью                  558
покупка коммерческой недвижимости         549
операции с жильем                         546
покупка жилья для сдачи                   544
жилье                                     535
операции с коммерческой недвижимостью     535
покупка своего жилья                      534
недвижимость                              533
покупка жилья для семьи                   528
строительство собственной недвижимости    528
операции со своей недвижимостью           526
покупка жилья                             523
строительство недвижимости                520
ремонт жилью                              515
покупка жилой недвижимости                514
покупка недвижимости                      510
строительство жилой недвижимости          507
на покупку своего автомобиля              428
заняться высшим образованием      

В столбце **education** присутствуют одинаковые значения разного регистра

In [25]:
data['education'] = data['education'].str.lower()
data['education'].value_counts()

среднее                12259
высшее                  4672
неоконченное высшее      703
начальное                187
ученая степень             4
Name: education, dtype: int64

In [26]:
print('Количество дубликатов:', data.duplicated().sum())
data[data.duplicated()]

Количество дубликатов: 71


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,2315,41,среднее,1,женат / замужем,0,F,сотрудник,0,151157,покупка жилья для семьи
3290,0,2315,58,среднее,1,гражданский брак,1,F,пенсионер,0,151157,сыграть свадьбу
4182,1,2315,34,высшее,0,гражданский брак,1,F,сотрудник,0,151157,свадьба
4851,0,2315,60,среднее,1,гражданский брак,1,F,пенсионер,0,151157,свадьба
5557,0,2315,58,среднее,1,гражданский брак,1,F,пенсионер,0,151157,сыграть свадьбу
6312,0,2315,30,среднее,1,женат / замужем,0,M,сотрудник,0,151157,строительство жилой недвижимости
7808,0,2315,57,среднее,1,гражданский брак,1,F,пенсионер,0,151157,на проведение свадьбы
7921,0,2315,64,высшее,0,гражданский брак,1,F,пенсионер,0,151157,на проведение свадьбы
7938,0,2315,71,среднее,1,гражданский брак,1,F,пенсионер,0,151157,на проведение свадьбы
8583,0,2315,58,высшее,0,Не женат / не замужем,4,F,пенсионер,0,151157,дополнительное образование


### Вывод

Для поиска и подсчета дупликатов-строк были использованы методы **duplicated()** и **sum()**<br>
По таблице, отфильтрованной методом **duplicated()**, можно сказать, что они появились из-за заполнения пропусков на предыдущем шаге, т.к. значения в столбцах **days_employed** и **total_income** совпадают со средним и медианным занчениями этих столбцов.  Эти строки можно оставить.<br>
Также данные строчного типа были приведены к одному регистру.<br>


### Лемматизация

In [27]:
data_purpose = data['purpose'].value_counts()
data_purpose

свадьба                                   668
на проведение свадьбы                     643
сыграть свадьбу                           634
операции с недвижимостью                  558
покупка коммерческой недвижимости         549
операции с жильем                         546
покупка жилья для сдачи                   544
жилье                                     535
операции с коммерческой недвижимостью     535
покупка своего жилья                      534
недвижимость                              533
покупка жилья для семьи                   528
строительство собственной недвижимости    528
операции со своей недвижимостью           526
покупка жилья                             523
строительство недвижимости                520
ремонт жилью                              515
покупка жилой недвижимости                514
покупка недвижимости                      510
строительство жилой недвижимости          507
на покупку своего автомобиля              428
заняться высшим образованием      

In [28]:
from pymystem3 import Mystem
from collections import Counter

m = Mystem()
words = []
for key, value in data_purpose.items():
    words += m.lemmatize(key)
Counter(words)

Counter({'свадьба': 3,
         '\n': 38,
         'на': 4,
         ' ': 59,
         'проведение': 1,
         'сыграть': 1,
         'операция': 4,
         'с': 5,
         'недвижимость': 10,
         'покупка': 10,
         'коммерческий': 2,
         'жилье': 7,
         'для': 2,
         'сдача': 1,
         'свой': 4,
         'семья': 1,
         'строительство': 3,
         'собственный': 1,
         'со': 1,
         'ремонт': 1,
         'жилой': 2,
         'автомобиль': 9,
         'заниматься': 2,
         'высокий': 3,
         'образование': 9,
         'сделка': 2,
         'подержанный': 1,
         'подержать': 1,
         'приобретение': 1,
         'дополнительный': 2,
         'получение': 3,
         'профильный': 1})

### Вывод

Лемматизацию целей кредита провели спомощью библиотеки **pymystem3**. Также посчитали частоту встречаемости разных слов с помощью класса **Counter**<br>
Из списка целей получения кредита можно выделить несколько обобщающих: <br>
Недвижимость, жилье, автомобиль, свадьба, образование - их можно использовать для категоризации данных

### Категоризация данных

In [29]:
purpose_dict = {}
general_purpos = ['недвижимость', 'автомобиль', 'образование', 'жилье', 'свадьба']
for key, value in data_purpose.items():
    for lem in m.analyze(key):
        if 'analysis' in lem:
            if lem['analysis'][0]['gr'][0] == 'S': #ищем существительные
                if lem['analysis'][0]['lex'] in general_purpos:
                    purpose_dict[key] = lem['analysis'][0]['lex']
purpose_dict

{'свадьба': 'свадьба',
 'на проведение свадьбы': 'свадьба',
 'сыграть свадьбу': 'свадьба',
 'операции с недвижимостью': 'недвижимость',
 'покупка коммерческой недвижимости': 'недвижимость',
 'операции с жильем': 'жилье',
 'покупка жилья для сдачи': 'жилье',
 'жилье': 'жилье',
 'операции с коммерческой недвижимостью': 'недвижимость',
 'покупка своего жилья': 'жилье',
 'недвижимость': 'недвижимость',
 'покупка жилья для семьи': 'жилье',
 'строительство собственной недвижимости': 'недвижимость',
 'операции со своей недвижимостью': 'недвижимость',
 'покупка жилья': 'жилье',
 'строительство недвижимости': 'недвижимость',
 'ремонт жилью': 'жилье',
 'покупка жилой недвижимости': 'недвижимость',
 'покупка недвижимости': 'недвижимость',
 'строительство жилой недвижимости': 'недвижимость',
 'на покупку своего автомобиля': 'автомобиль',
 'заняться высшим образованием': 'образование',
 'сделка с подержанным автомобилем': 'автомобиль',
 'свой автомобиль': 'автомобиль',
 'автомобиль': 'автомобиль',


In [30]:
data['general_purpose'] = data['purpose'].apply(lambda purpose: purpose_dict[purpose])
data['general_purpose'].value_counts()

недвижимость    5280
жилье           3725
автомобиль      3553
образование     3322
свадьба         1945
Name: general_purpose, dtype: int64

In [31]:
data['has_child'] = data['children'].apply(lambda children: True if children > 0 else False)
data['has_child'].value_counts()

False    10846
True      6979
Name: has_child, dtype: int64

In [32]:
def count_children(children):
    if children == 0:
        return 'Нет детей'
    if children > 2:
        return 'Многодетная семья'
    return '1 или 2 ребенка'

data['count_child'] = data['children'].apply(count_children)
data['count_child'].value_counts()

Нет детей            10846
1 или 2 ребенка       6541
Многодетная семья      438
Name: count_child, dtype: int64

In [33]:
lowest_treshold = data['total_income'].max() / 5
low_treshold = lowest_treshold * 2
high_treshold = lowest_treshold * 3
highest_treshold = lowest_treshold * 4
def set_level_income(total_income):
    if total_income <= lowest_treshold:
        return 'Очень низкий доход'
    if lowest_treshold < total_income <= low_treshold:
        return 'Низкий доход'
    if low_treshold < total_income <= high_treshold:
        return 'Средний доход'
    if high_treshold < total_income <= highest_treshold:
        return 'Высокий доход'
    if highest_treshold < total_income:
        return 'Очень высокий доход'
data['total_income'].apply(set_level_income).value_counts()

Очень низкий доход     17530
Низкий доход             266
Средний доход             21
Высокий доход              6
Очень высокий доход        2
Name: total_income, dtype: int64

Как можно заметить, категоризация уровня дохода методом выше не подходит из-за неравномерного распределения. 

In [34]:
lowest_treshold = data['total_income'].quantile(.2)
low_treshold = data['total_income'].quantile(.4)
high_treshold = data['total_income'].quantile(.6)
highest_treshold = data['total_income'].quantile(.8)
def set_level_income(total_income):
    if total_income <= lowest_treshold:
        return 'Очень низкий доход'
    if lowest_treshold < total_income <= low_treshold:
        return 'Низкий доход'
    if low_treshold < total_income <= high_treshold:
        return 'Средний доход'
    if high_treshold < total_income <= highest_treshold:
        return 'Высокий доход'
    if highest_treshold < total_income:
        return 'Очень высокий доход'
data['level_income'] = data['total_income'].apply(set_level_income)
data['level_income'].value_counts()

Очень низкий доход     3565
Очень высокий доход    3565
Низкий доход           3565
Высокий доход          3565
Средний доход          3565
Name: level_income, dtype: int64

In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17825 entries, 0 to 21524
Data columns (total 16 columns):
children            17825 non-null int64
days_employed       17825 non-null int64
dob_years           17825 non-null int64
education           17825 non-null object
education_id        17825 non-null int64
family_status       17825 non-null object
family_status_id    17825 non-null int64
gender              17825 non-null object
income_type         17825 non-null object
debt                17825 non-null int64
total_income        17825 non-null int64
purpose             17825 non-null object
general_purpose     17825 non-null object
has_child           17825 non-null bool
count_child         17825 non-null object
level_income        17825 non-null object
dtypes: bool(1), int64(7), object(8)
memory usage: 2.2+ MB


In [36]:
education_dict = data[['education', 'education_id']].drop_duplicates().reset_index(drop=True)
family_status_dict = data[['family_status', 'family_status_id']].drop_duplicates().reset_index(drop=True)
data_log = data[['children', 'days_employed', 'dob_years', 'education_id', 'family_status_id', 'gender', 'income_type', 'debt', 'total_income', 'general_purpose', 'has_child']]
data_log.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,general_purpose,has_child
0,1,8437,42,0,0,F,сотрудник,0,253875,жилье,True
1,1,4024,36,1,0,F,сотрудник,0,112080,автомобиль,True
2,0,5623,33,1,0,M,сотрудник,0,145885,жилье,False
3,3,4124,32,1,0,M,сотрудник,0,267628,образование,True
5,0,926,27,0,1,M,компаньон,0,255763,жилье,False
6,0,2879,43,0,0,F,компаньон,0,240525,жилье,False
7,0,152,50,1,0,M,сотрудник,0,135823,образование,False
8,2,6929,35,0,1,F,сотрудник,0,95856,свадьба,True
9,0,2188,41,1,0,M,сотрудник,0,144425,жилье,False
10,2,4171,36,0,0,M,компаньон,0,113943,недвижимость,True


In [37]:
education_dict

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [38]:
family_status_dict

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,в разводе,3
3,Не женат / не замужем,4
4,вдовец / вдова,2


### Вывод

Для более компактного вывода в словари были вынесены столбцы **education** и **family_status**, т.к. у них уже есть столбцы с соответствующим им идентификаторами **education_id** и **family_status_id** *(или надо все строковые столбцы перевести в столбцы???)*<br>
<br>
Также данные были категоризированны еще по 4 пунктам:<br>
1) Более обощенная цель взятия заема **general_purpose**, т.к. в большинстве случаев имеют один смысл, но разные формулировки<br>
2) Наличие детей **has_child**<br>
3) Размер семьи **count_child**. Наличие детей и размер семьи - нужны для группировки по этим полям и поиска зависимости с задолжностями в дальнейшем<br>
4) Размер дохода **level_income**. Также как и предыдущие поля нужны, чтобы потом можно было сделать группировку. С числовыми значениями так не получится, потому что почти все они имеют уникальное значения, поэтому разбили на диапазоны. В качестве разбиения использовали квантили, потому что распределение доходов неравномерное

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [56]:
data_children_group = round(data.groupby('has_child')['debt'].mean() * 100, 2)
data_children_group.sort_values(ascending=False)

has_child
True     9.41
False    8.16
Name: debt, dtype: float64

In [57]:
data_children_group = round(data.groupby('count_child')['debt'].mean() * 100, 2)
data_children_group.sort_values(ascending=False)

count_child
1 или 2 ребенка      9.46
Многодетная семья    8.68
Нет детей            8.16
Name: debt, dtype: float64

### Вывод

Чем меньше детей, тем меньше вероятность заодлжности. Очевидно, что дети добавляют расходы

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [58]:
data_fam_status_group = round(data.groupby('family_status')['debt'].mean() * 100, 2)
data_fam_status_group.sort_values(ascending=False)

family_status
Не женат / не замужем    10.50
гражданский брак         10.02
женат / замужем           7.96
в разводе                 7.36
вдовец / вдова            6.58
Name: debt, dtype: float64

### Вывод

Люди не женатые/ не замужние с большей вероятностью могут опоздать с выплатой долга. Вероятно это связанно с тем, что живущие одни люди должны все полностью выплачивать сами. А те, кто состоят в браке, могут совместно делать выплаты. Те, кто были в браке, а теперь нет, вероятно имеют накопления. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [59]:
data_purpose_group = round(data.groupby('level_income')['debt'].mean() * 100, 2)
data_purpose_group.sort_values(ascending=False)

level_income
Низкий доход           9.57
Очень низкий доход     9.12
Высокий доход          8.95
Средний доход          8.42
Очень высокий доход    7.21
Name: debt, dtype: float64

### Вывод

Очевидно, что чем ниже доход, тем выше вероятность задолжности

- Как разные цели кредита влияют на его возврат в срок?

In [60]:
data_purpose_group = round(data.groupby('general_purpose')['debt'].mean() * 100, 2)
data_purpose_group.sort_values(ascending=False)

general_purpose
автомобиль      9.96
образование     9.84
свадьба         8.38
недвижимость    8.05
жилье           7.33
Name: debt, dtype: float64

### Вывод

Вероятность задолжности по автомобилям больше всех, а жилью меньше всех. Возможно связано с тем, что время выплаты
на жилье дается больше.

### Шаг 4. Общий вывод

In [54]:
data_pivot = data.pivot_table(index=['level_income'], columns='family_status', values='debt', aggfunc=['mean'])
#for col in data_pivot:
#    data_pivot[col[0]][col[1]] = data_pivot[col[0]][col[1]] * 100
round(data_pivot * 100, 2)

mean                           \
family_status       Не женат / не замужем в разводе вдовец / вдова   
level_income                                                         
Высокий доход                       14.49      7.28           9.72   
Низкий доход                        11.13      8.06           7.69   
Очень высокий доход                  9.81      6.12           6.85   
Очень низкий доход                   9.19      8.21           2.91   
Средний доход                        7.68      7.18           6.82   

                                                      
family_status       гражданский брак женат / замужем  
level_income                                          
Высокий доход                   9.81            7.53  
Низкий доход                   10.38            9.08  
Очень высокий доход             7.20            6.74  
Очень низкий доход             10.84            8.89  
Средний доход                  11.72            7.63

Можно выделить два крайних случая: Женатый/замужний (или был таким) человек без детей с высоким доходом, покупающий жилье, более надежен, чем холостой с детьми с низким доходом, покупающий автомобиль. <br>
Зависимость от количества детей не такая сильная. Более важными показателями являются семейный статус и уровень дохода